# Challenge for Data Engineer position at TI

Borja González Ferreiro


# First exercise: count the number of lines in Python for each file

In [1]:
bookingName = "../bookings.csv"
searchName = "../searches.csv" 

with open(bookingName, "r") as f: nbLinesBook = sum(1 for line in f)
with open(searchName, "r")  as f: nbLinesSearch = sum(1 for line in f)
    
print("Number of lines for {}: {:n} (header included)".format(bookingName, nbLinesBook))
print("Number of lines for {}: {:n} (header included)".format(searchName, nbLinesSearch))

Number of lines for ../bookings.csv: 10000011 (header included)
Number of lines for ../searches.csv: 20390199 (header included)


# Second exercise: top 10 arrival airports in the world in 2013 (using the bookings file)

In [2]:
import pandas as pd # For bonus points :)


# Since the data fits in memory, it is not necessary to process the file by chunks
# We are only going to use columns arr_port and pax, so not need to load the rest
bookings = pd.read_csv(bookingName,  sep='^', usecols=["arr_port", "pax"])# dtype={'pax':int, 'arr_port':str})

### Quick check that there are no oddities in the data such as missing values

In [3]:
bookings.describe()

,pax
count,10000009.000000
mean,0.490880
std,2.199173
min,-90.000000
25%,-1.000000
50%,1.000000
75%,1.000000
max,99.000000


We notice that there is something off since there should be 10000010 instead of 10000009. Let's find which row contain an NaN value and remove it from our dataframe:

In [5]:
bookings=bookings[bookings.pax.notnull()]

### top 10 arrival airports in the world in 2013

In [11]:
groupAir = bookings[['arr_port', 'pax']].groupby('arr_port', sort=False)

numAir = 10
airTopPax=groupAir.sum().nlargest(numAir, 'pax')

In [14]:
from GeoBases import GeoBase
geo_o = GeoBase(data='ori_por', verbose=False)

#Add column with the city corresponding to the airport
airTopPax['City'] = airTopPax.apply(lambda row: geo_o.get(row.name.strip(),'city_name_ascii'), axis=1)

# Change names of columns and index for display
airTopPax.rename(columns={ 'pax': 'Arrivals'}, inplace=True)
airTopPax.index.names = ['Airport']

print("The {} airports with more passengers are:".format(numAir))
airTopPax

The 10 airports with more passengers are:


,Arrivals,City
Airport,,
LHR,88809,London
MCO,70930,Orlando
LAX,70530,Los Angeles
LAS,69630,Las Vegas
JFK,66270,New York City
CDG,64490,Paris
BKK,59460,Bangkok
MIA,58150,Miami
SFO,58000,San Francisco


# Third exercise: plot the monthly number of searches for flights arriving at Málaga, Madrid or Barcelona¶

After taking a look at the data, some lines use as delimiter between fields a ','. That is why I modified the original searches file in order to have exclussively ^ as delimiter. I used the following command:
sed 's/,/\^/g' ../searches.csv > ../correctedSearches.csv

In [2]:
import pandas as pd # For bonus points :)


searchName = "../correctedSearches.csv" 

# Since the data fits in memory, it is not necessary to process the file by chunks
# We are only going to use columns Origin and Destination, so not need to load the rest
# Taking a look at the file, we notice most of the lines use ^ as field separator and only 33 lines use ,
searches = pd.read_csv(searchName,  sep='^', usecols=["Date", "Origin", "Destination"], parse_dates=['Date'], dayfirst=False, index_col='Date')

### Quick check that there are no oddities in the data such as missing values¶

In [3]:
searches.describe()

,Origin,Destination
count,20390198,20390198
unique,2503,3068
top,FRA,BKK
freq,550736,477093


We notice that the number of values for Origin and destination match the number of lines in the file (without the header).

### Plotting the monthly number of searches for flights arriving at Málaga, Madrid or Barcelona

First, we pick only the rows associated with destinations: Madrid, Málaga and Barcelona airports. Then we group the values  by month and destination and we count the number of searches for each destination, so that finally I only have to plot them. 

In [8]:
airSearches = searches[(searches["Destination"] == "MAD") | (searches["Destination"] == "BCN") | (searches["Destination"] == "AGP")]

monthSearches = airSearches.groupby([pd.TimeGrouper(freq='M'), "Destination"]).count()
